In [1]:
import resources as r

import ipywidgets as widgets
from IPython.display import HTML, display

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget-inline-hbox { min-width: 50ex !important; }
    .widget-button { min-width: max-content }
</style>'''))
layout = widgets.Layout(width='50%')

fluorophores, solvents, methods = r.fluorophores_solvents_methods()
basisOptions = r.Basis
methodOptions = r.Methods

def submitORCA(memory:int, cores:int, partner:bool, jobType:r.Jobs, stateList:list[r.States], grid:r.Grids, basisList:list[r.Basis], 
               methodList:list[r.Methods], solventList:list[r.Solvents], 
               fluorophoreList:list[r.Fluorophores], pcm:r.PCM, pcmEq:r.PCM.Eq, useXD: bool, useBest:bool, submit:bool, resubmit:bool) -> None:
    with r.statusLoad('progress_s1') as df_s1:
        with r.statusLoad('progress_s0') as df_s0:
            with r.statusLoad('progress_s2') as df_s2:
                dfDict = {r.States.s0: df_s0, r.States.s1: df_s1, r.States.s2: df_s2}
                with r.monarchHandler() as mon:
                    for fluorophore in fluorophoreList:
                        for method in methodList:
                            for solvent in solventList:
                                for basis in basisList:
                                    for state in stateList:
                                        xyzfile = ''
                                        mofile = ''
                                        scfstring = ''
                                        kdiis = False
                                        soscf = True
                                        notrah = False

                                        status       = dfDict[state      ].at[(fluorophore, method          ), solvent]
                                        status_gs    = dfDict[r.States.s0].at[(fluorophore, method          ), solvent]
                                        status_gs_xd = dfDict[r.States.s0].at[(fluorophore, r.Methods.wb97xd), solvent]
                                        status_state_xd = dfDict[state      ].at[(fluorophore, r.Methods.wb97xd), solvent]

                                        xd_es_refjob = r.Job(r.Software.orca,  fluorophore, solvent, r.Methods.wb97xd, r.Basis.augccpvdz, r.PCM.smd,  r.PCM.Eq.eq,   state,       r.Jobs.opt)
                                        xd_gs_refjob = r.Job(r.Software.orca,  fluorophore, solvent, r.Methods.wb97xd, r.Basis.augccpvdz, r.PCM.smd,  r.PCM.Eq.eq,   r.States.s0, r.Jobs.opt)
                                        gs_refjob    = r.Job(r.Software.orca,  fluorophore, solvent, method,           r.Basis.augccpvdz, r.PCM.smd,  r.PCM.Eq.eq,   r.States.s0, r.Jobs.opt)
                                        crest_refjob = r.Job(r.Software.crest, fluorophore, solvent, r.Methods.crest,  r.Basis.none,      r.PCM.alpb, r.PCM.Eq.none, r.States.s0, r.Jobs.crest)

                                    if status in [r.Status.SMD.failed, None] or resubmit:
                                        if useBest:
                                            if useXD and status_state_xd == r.Status.SMD.finished:
                                                refJob = xd_es_refjob
                                                print('Using ωB97X-D ES geom')
                                            elif useXD and status_gs_xd == r.Status.SMD.finished:
                                                refJob = xd_gs_refjob
                                                print('Using ωB97X-D GS geom')
                                            elif status_gs == r.Status.SMD.finished:
                                                refJob = gs_refjob
                                            else:
                                                refJob = crest_refjob
                                                print('Using CREST geom')

                                        elif useXD:
                                            if status_state_xd == r.Status.SMD.finished:
                                                refJob = xd_es_refjob
                                            elif status_gs_xd == r.Status.SMD.finished:
                                                refJob = xd_gs_refjob
                                
                                        else:
                                            if status_gs == r.Status.SMD.finished:
                                                refJob = gs_refjob

                                        job = r.Job(r.Software.orca, fluorophore, solvent, method, basis, pcm, pcmEq, state, jobType, grid=grid,
                                                    mopath=mofile, xyzpath=xyzfile, catxyzpath=refJob.xyzfile, kdiis=kdiis, soscf=soscf, notrah=notrah, scfstring=scfstring,
                                                    submit=submit, submitFlags=f'-NSs', partner=partner, procs=cores, mem=memory)

                                        fileName = mon.buildJob(job)

                                        print(fileName)
                                        if submit:
                                            dfDict[state].at[(fluorophore, method), solvent] = r.Status.SMD.queued
                                    elif status == r.Status.SMD.finished:
                                        print(f'{fluorophore.name} Orca job already finished, use "resubmit" to overwrite the job file')

                                    elif status in [r.Status.SMD.queued, r.Status.SMD.running]:    
                                        print(f'{fluorophore.name} Orca job already submitted, use "resubmit" to overwrite the job file')

                                    else:
                                        raise Exception("This case has not been accounted for, Adrea...")
    print('Done!')

widgets.interactive(submitORCA, {'manual' : True, 'manual_name' : 'Build ORCA Job'},
                     jobType=widgets.Dropdown(options=r.Jobs, rows=6, description='Job Type'),
                     fluorophoreList=widgets.SelectMultiple(options=fluorophores, rows=12, description='Fluorophore'),
                     basisList=widgets.SelectMultiple(options=r.Basis, rows=15, description='Basis'),
                     grid=widgets.Dropdown(options=r.Grids, rows=15, description='Grids', value=r.Grids.g75_302),
                     pcm=widgets.Dropdown(options=r.PCM, rows=5, description='PCM', value=r.PCM.smd),
                     pcmEq=widgets.Dropdown(options=r.PCM.Eq, rows=5, description='PCM Eq'),
                     solventList=widgets.SelectMultiple(options=solvents, rows=12, description='Solvents'),
                     methodList=widgets.SelectMultiple(options=r.Methods, rows=10, description='Method'),
                     stateList=widgets.SelectMultiple(options=r.States, rows=4, description='States'),
                     partner = widgets.Checkbox(value=True, description='Partner'),
                     useBest = widgets.Checkbox(value=True, description='Use S0 if ES not ready', layout=layout),
                     useXD = widgets.Checkbox(value=True, description='Use ωB97X-D opt for other functionals', layout=layout),
                     submit = widgets.Checkbox(value=False, description='Submit'),
                     resubmit = widgets.Checkbox(value=False, description='ReSubmit'),
                     memory = widgets.IntText(value=64, description='Memory'),
                     cores = widgets.IntText(value=16, description='Cores'),
                     )

interactive(children=(IntText(value=64, description='Memory'), IntText(value=16, description='Cores'), Checkbo…